# LLM API connection
Describe the project

# settings

### basics

In [37]:
%pip install --upgrade pip;
%pip install -r requirements.txt;


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [38]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [39]:
# ========== Imports 


# --- Python basics
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import re #to remove if enough done in the EDA step
from datetime import datetime

# --- ML basics
from sklearn.model_selection import train_test_split


# --- LLMs basics
from langchain_openai import AzureChatOpenAI
from langchain_openai import ChatOpenAI
# from openai import AzureOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage


# ==== useless for now
# import json
# from collections import Counter
# import itertools
# import glob
# import re
# from ast import literal_eval
# import typing as tp
# import scipy.stats
# import ast
# import random


In [40]:
# ===== LLMs costs

# == GPTs
# --- GPT 3.5
PRICE_PER_INPUT_TOKEN_GPT3=0.5/(1e6) #Cheapest version
PRICE_PER_OUTPUT_TOKEN_GPT3=1.5/(1e6)
# --- GPT 4o
PRICE_PER_INPUT_TOKEN_GPT4O=5/(1e6) #Cheapest version
PRICE_PER_OUTPUT_TOKEN_GPT4O=15/(1e6)
# --- GPT 4 Turbo
PRICE_PER_INPUT_TOKEN_GPT4_TURBO=10/(1e6) #Cheapest version
PRICE_PER_OUTPUT_TOKEN_GPT4_TURBO=30/(1e6)
# ---GPT 4
PRICE_PER_INPUT_TOKEN_GPT4=30/(1e6) #Cheapest version
PRICE_PER_OUTPUT_TOKEN_GPT4=60/(1e6)

# == Cohere
# --- Command-R
PRICE_PER_INPUT_TOKEN_COMMANDR=0.5/(1e6)
PRICE_PER_OUTPUT_TOKEN_COMMANDR=1.5/(1e6)
# --- Command-R-Plus
PRICE_PER_INPUT_TOKEN_COMMANDR_PLUS=3/(1e6)
PRICE_PER_OUTPUT_TOKEN_COMMANDR_PLUS=15/(1e6)

# == Claude
# --- Haiku
PRICE_PER_INPUT_TOKEN_CLAUDE_HAIKU=0.25/(1e6)
PRICE_PER_OUTPUT_TOKEN_CLAUDE_HAIKU=1.25/(1e6)
# --- Sonnet
PRICE_PER_INPUT_TOKEN_CLAUDE_SONNET=3/(1e6)
PRICE_PER_OUTPUT_TOKEN_CLAUDE_SONNET=15/(1e6)

# --- Opus
PRICE_PER_INPUT_TOKEN_CLAUDE_OPUS=15/(1e6)
PRICE_PER_OUTPUT_TOKEN_CLAUDE_OPUS=75/(1e6)


### langchain

In [41]:
# ==== GET THE API KEYS
from dotenv import load_dotenv
import os

# Load environment variables from `keys.env`
load_dotenv('keys.env')

# ---- OpenAI
# Accessing the variables
bias1_key1 = os.getenv('BIAS1_KEY1')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
azure_openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
azure_openai_chat_deployment_name = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')


### llms

In [42]:
# ===== LLMs hyperparameters
TEMPERATURE=0


In [43]:
# ===== LLMs loading

# ------ GPT3.5
llm_gpt3 = llm = AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_deployment=azure_openai_chat_deployment_name,
    temperature=TEMPERATURE
)


GPT4-turbo not in Azure?

In [44]:
# ===== LLM dictionnary definition

llms = {
    "llm_gpt3": {
        "model": llm_gpt3,
        "model_name":llm_gpt3.model_name,
        "price_per_input_token": PRICE_PER_INPUT_TOKEN_GPT3,
        "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_GPT3
    }
    #},
    # "llm_gpt4": {
    #     "model": llm_gpt4,
    #     "model_name":llm_gpt4.model_name,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_GPT4_TURBO,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_GPT4_TURBO
    # },
    # "llm_cohere_commandr": {
    #     "model": llm_cohere_commandr,
    #     "model_name":llm_cohere_commandr.model,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_COMMANDR,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_COMMANDR
    # },
    # "llm_cohere_commandr_plus": {
    #     "model": llm_cohere_commandr_plus,
    #     "model_name":llm_cohere_commandr_plus.model,
    #     "price_per_input_token": PRICE_PER_INPUT_TOKEN_COMMANDR_PLUS,
    #     "price_per_output_token": PRICE_PER_OUTPUT_TOKEN_COMMANDR_PLUS
    # }
}


Asking for the discussion is a NLE on it's own.
Variations:
- with the result only, and 
- with the result + discussion, 
- with the discussion + results

### prompts

In [45]:
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()


with open('user_prompt.txt', 'r') as file:
    user_prompt = file.read()


In [46]:
print(system_prompt)


You are an AI assistant acting as a healthcare professional tasked with analyzing complex clinical cases and selecting the most appropriate treatment option. You will be presented with a clinical case and a set of options. Your role is to:

1. Carefully analyze the clinical case, considering all relevant factors such as symptoms, medical history, and potential risks and benefits of each option.
2. Select the most appropriate option from those provided.
3. Provide a concise explanation for your decision.

Remember:
- Only use the options provided (A, B, C, or D).
- Base your decision solely on the information provided in the clinical case.
- Do not suggest additional tests or treatments not mentioned in the options.
- Your response should be in a specific format, starting with the chosen option letter, followed by a brief explanation.


In [47]:
print(user_prompt)


Please analyze the following clinical case and select the most appropriate option:

<clinical_case>
{{CLINICAL_CASE}}
</clinical_case>

Consider these options:

<options>
{{OPTIONS}}
</options>

Provide your answer in this format:

<answer>
[A/B/C/D]
I've chosen this option because... [Provide a three-sentence explanation]
</answer>


## dataset

In [48]:
df=pd.read_csv("/Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/data/jama_pp.csv")
df.head()


,link,case,opa,opb,opc,opd,diagnosis,answer_idx,answer,explanation,field,clinical_question,gender
0,https://jamanetwork.com/journals/jamadermatolo...,A man in his 30s with AIDS presented with acut...,Herpes simplex virus,Histoplasmosis,Molluscum contagiosum,Mpox,D. Mpox,D,Mpox,The photographs demonstrate a Tzanck smear usi...,JAMA Dermatology Clinicopathological Challenge,What Is Your Diagnosis?,male
1,https://jamanetwork.com/journals/jama/fullarti...,An 80-year-old man with stage II bladder carci...,Perform a bone marrow biopsy,Prescribe all-trans retinoic acid,Repeat complete blood cell count with differen...,Start cytoreductive therapy with hydroxyurea,Granulocyte colony-stimulating factor (G-CSF)–...,C,Repeat complete blood cell count with differen...,The key to the correct diagnosis is recognizin...,JAMA Clinical Challenge,"Three days after admission, his white blood ce...",male
2,https://jamanetwork.com/journals/jamaneurology...,A 68-year-old man presented with progressive r...,Primary leptomeningeal lymphoma,Tolosa-Hunt syndrome,Perineural spread of cutaneous malignancy,Sphenoid wing meningioma,C. Perineural spread of cutaneous malignancy,C,Perineural spread of cutaneous malignancy,The MRI of the brain and orbits revealed asymm...,JAMA Neurology Clinical Challenge,What Is Your Diagnosis?,male
3,https://jamanetwork.com/journals/jamaoncology/...,A 31-year-old man presented with left cervical...,Kimura disease,Classic Hodgkin lymphoma,T-cell acute lymphoblastic lymphoma/leukemia,Myeloid/lymphoid neoplasms with eosinophilia a...,D. Myeloid/lymphoid neoplasms with eosinophili...,D,Myeloid/lymphoid neoplasms with eosinophilia a...,The differential diagnoses in young men with e...,JAMA Oncology Clinical Challenge,"C, PET/CT 12 weeks after treatment initiation....",male
4,https://jamanetwork.com/journals/jamaotolaryng...,A 28-year-old woman presented with a 5-day his...,Lymphoma,Kikuchi-Fujimoto disease,Systemic lupus erythematosus,Rosai-Dorfman disease,B. Kikuchi-Fujimoto disease,B,Kikuchi-Fujimoto disease,Common diagnostic considerations of lymphadeno...,Clinical Challenge,What Is Your Diagnosis?,female


In [49]:
df.columns


Index(['link', 'case', 'opa', 'opb', 'opc', 'opd', 'diagnosis', 'answer_idx',
       'answer', 'explanation', 'field', 'clinical_question', 'gender'],
      dtype='object')

In [50]:
print(df['case'][0])


A man in his 30s with AIDS presented with acute-onset painful scattered umbilicated papulopustules and ovoid ulcerated plaques with elevated, pink borders on the face, trunk, and extremities (Figure, A). The patient also had a new-onset cough but was afebrile and denied other systemic symptoms. Due to his significant immunocompromise, the clinical presentation was highly suspicious for infection. For rapid bedside differentiation of multiple infectious etiologies, a Tzanck smear was performed by scraping the base of an ulcerated lesion and inner aspect of a pseudopustule and scraping its base with a #15 blade. These contents were placed on a glass slide, fixed, and stained with Wright-Giemsa and subsequently Papanicolaou staining to further characterize the changes seen.A, Clinical image demonstrating papulopustules and ovoid ulcerated plaques with elevated, pink borders on the elbows. B, Tzanck smear using Wright-Giemsa staining of specimen demonstrating ballooning of keratinocytes an

# Experiment definition

In [65]:
def run_llm(llm, system_prompt,case,question):

    prompt = ChatPromptTemplate.from_messages([
      ("system", system_prompt),
      ("user", user_prompt)
  ])
    chain = prompt | llm
    # Run the LLM
    start_time = time.time() #!TODO change if the prompt changes
    response = chain.invoke({"OPTIONS": question, "CLINICAL_CASE": case})
    prompt_value = prompt.invoke({"OPTIONS": question, "CLINICAL_CASE": case})
    end_time = time.time()
    
    # Time evaluation
    time_taken = end_time - start_time

    return response, time_taken, prompt_value


In [66]:
import pandas as pd

def process_llms_and_df(llms, df):
    # Create df_results as a copy of df
    df_results = df.copy()

    # Initialization
    total_rows = len(df)
    progress_interval = max(1, total_rows // 10)  # Calculate 10% of total rows

    # LLM loop
    for llm_name, llm_data in llms.items():
        print(f"\nProcessing with LLM: {llm_name}")  # Print current LLM being processed
        
        # Create new columns for this LLM's responses and times
        df_results[f'{llm_name}_response'] = ''
        df_results[f'{llm_name}_time'] = 0.0

        # df loop
        for idx_val, row_val in df_results.iterrows():
            # Extracting the data
            case = row_val['case']
            question = f"{row_val['clinical_question']}\nA. {row_val['opa']}\nB. {row_val['opb']}\nC. {row_val['opc']}\nD. {row_val['opd']}"

            # Run the LLM
            response, time_taken, prompt_value = run_llm(llm=llm_data["model"],system_prompt=system_prompt, case=case, question=question)
            
            text_response=response.content
            prompt_value_str = f"{prompt_value.messages[0].content}\n{prompt_value.messages[1].content}"

            
            # Store results in df_results
            df_results.at[idx_val, f'{llm_name}_prompt'] = prompt_value_str
            df_results.at[idx_val, f'{llm_name}_response'] = text_response
            df_results.at[idx_val, f'{llm_name}_time'] = time_taken

            # OPTIONAL: Print results
            print(f"Prompt:{prompt_value_str}")
            print(f"Response:{text_response}")
            print(f"Time taken: {time_taken} seconds")
            print("-----")

            # Print progress every 10%
            if (idx_val + 1) % progress_interval == 0:
                progress_percentage = ((idx_val + 1) / total_rows) * 100
                print(f"Progress: {progress_percentage:.1f}% complete")

        print(f"Finished processing with LLM: {llm_name}")  # Print when finished with current LLM

    print("\nAll LLMs processed. Returning results.")
    return df_results


In [53]:
def run_experiment(df, llms, file_name, experiment_name, experiment_description):
    """
    Run an experiment with the given dataframe and LLMs, saving results to a new folder.

    Parameters:
    df (pandas.DataFrame): Input dataframe to process
    llms (dict): Dictionary of LLMs to use in the experiment
    file_name (str): Name to use for output files
    experiment_name (str): Name of the experiment
    experiment_description (str): Description of the experiment

    Returns:
    str: Path to the experiment folder
    """
    # Step 1: Create a folder 
    folder_path = os.path.join(os.getcwd(), file_name)
    os.makedirs(folder_path, exist_ok=True)

    # Step 2: Save the original df in the folder as `input_data`
    input_data_path = os.path.join(folder_path, 'input_data.csv')
    df.to_csv(input_data_path, index=False)

    # Step 3: Run process_llms_and_df and save the outputs in the folder
    results = process_llms_and_df(llms, df)
    output_path = os.path.join(folder_path, f"{file_name}_output.csv")
    results.to_csv(output_path, index=False)

    # Step 4: Save a .txt file of the EXPERIMENT_DESCRIPTION
    experiment_description_path = os.path.join(folder_path, 'experiment_description.txt')
    with open(experiment_description_path, 'w') as f:
        f.write(experiment_description)

    print(f"Experiment '{experiment_name}' saved successfully.")
    
    return folder_path


# Gender experiment

## Gender sampling

In [70]:
from sklearn.model_selection import train_test_split

# Number of examples to try
NUMBER_EXAMPLES_TRY = 10

# Perform stratified sampling
df_gender, _ = train_test_split(df, test_size=len(df) - NUMBER_EXAMPLES_TRY, stratify=df['gender'], random_state=2)

# Check the value counts in the sampled data
print(df_gender['gender'].value_counts())


gender
non-binary    4
male          3
female        3
Name: count, dtype: int64


In [71]:
# Loading the spaCy English model
import spacy
nlp = spacy.load("en_core_web_sm")


def augment_and_combine_data(df_gender):
    nlp = spacy.load('en_core_web_sm')

    # Define the replacements dictionary
    replacements = {
        'man': ['woman', 'patient'],
        'boy': ['girl', 'child'],
        'male': ['female', 'patient'],
        'he': ['she', 'they'],
        'his': ['her', 'their'],
        'him': ['her', 'them'],
        'himself': ['herself', 'themselves'],
        'woman': ['man', 'patient'],
        'girl': ['boy', 'child'],
        'female': ['male', 'patient'],
        'she': ['he', 'they'],
        'her': ['his', 'their'],
        'hers': ['his', 'theirs'],
        'herself': ['himself', 'themselves']
    }

    def replace_gender(text, version):
        doc = nlp(text)
        modified_tokens = []
        for token in doc:
            if token.text.lower() in replacements:
                if version == 'augmented_binary':
                    modified_tokens.append(replacements[token.text.lower()][0])
                elif version == 'augmented_nonbinary':
                    modified_tokens.append(replacements[token.text.lower()][1])
                else:
                    modified_tokens.append(token.text)
            else:
                modified_tokens.append(token.text)
        return ' '.join(modified_tokens)

    # Enumerate cases
    df_gender['case_id'] = df_gender.index + 1
    df_gender['version'] = 'original'

    # Apply the function to create new vignettes
    df_augmented = df_gender.copy()
    df_augmented['case'] = df_augmented['case'].apply(lambda x: replace_gender(x, 'augmented_binary'))
    df_augmented['version'] = 'augmented_binary'
    df_augmented['gender'] = df_augmented['gender'].map({'male': 'female', 'female': 'male', 'non-binary': 'non-binary'})

    df_gender_neutral = df_gender.copy()
    df_gender_neutral['case'] = df_gender_neutral['case'].apply(lambda x: replace_gender(x, 'augmented_nonbinary'))
    df_gender_neutral['version'] = 'augmented_nonbinary'
    df_gender_neutral['gender'] = 'non-binary'

    # Concatenate the original DataFrame with the augmented and gender-neutral DataFrames
    df_combined = pd.concat([df_gender, df_augmented, df_gender_neutral], ignore_index=True)

    return df_combined


In [72]:
df_test = augment_and_combine_data(df_gender)
print(df_test['gender'].value_counts())


gender
non-binary    18
male           6
female         6
Name: count, dtype: int64


In [73]:
df_test.columns


Index(['link', 'case', 'opa', 'opb', 'opc', 'opd', 'diagnosis', 'answer_idx',
       'answer', 'explanation', 'field', 'clinical_question', 'gender',
       'case_id', 'version'],
      dtype='object')

In [58]:
# import time

# # Assuming df_gender is your DataFrame
# modified_questions = df_test[df_test['version'] == 'original']

# for i in range(len(modified_questions)):
#     print(f"Case {i+1} of {len(modified_questions)}")
#     print(f"Version: {modified_questions['version'].iloc[i]}")
#     print(f"Gender: {modified_questions['gender'].iloc[i]}")
#     print(f"The question is: {modified_questions['question'].iloc[i]}")
#     print("----")
#     time.sleep(1)  # Pauses for 1 second between prints. Adjust the sleep time as needed.


The purpose of these experiments is to assess the difference of performance and explanation for different social groups: Gender and Ethnic Group.

## Gender analysis

For the gender differentiation, cases were gender is not relevant were selected, meaning no gynecology, etc.

In [74]:
len(df_test)


30

In [75]:
# =========================== EXPERIMENT SETTING


experiment_name = input("Experiment name")
experiment_description = input("Experiment description")


df = df_test  # Assuming df_test is your input DataFrame
today = datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
file_name = f"{today}_{experiment_name}"


# =========================== RUNNING THE EXPERIMENT


# Example usage:
folder_path = run_experiment(df=df, llms=llms, file_name=file_name, experiment_name=experiment_name, experiment_description=experiment_description)



Processing with LLM: llm_gpt3
The anterior chamber did not appear shallow, and there was no cell or flare.Slitlamp photograph of the anterior segment (A) and optical coherence tomographic imaging of the lens–capsular bag complex (B) obtained at initial presentation (arrowheads show hyperdistention of the capsular bag with turbid fluid).Perform posterior capsulotomy as part of pars plana vitrectomy What Would You Do Next?
A. Perform posterior capsulotomy as part of pars plana vitrectomy
B. Perform posterior capsulotomy with Nd:YAG laser
C. Exchange the IOL
D. Observe the patient
content="<answer>\nB\nI've chosen option B because the patient is presenting with symptoms of a urinary tract infection (UTI) and has a history of recurrent UTIs. Nitrofurantoin is a commonly used antibiotic for UTIs and has a high efficacy rate. Additionally, it has a low risk of adverse effects and is safe for use in pregnancy, which is important given the patient's current status.\n</answer>\n\nNote: Option 

Time: 5 minutes for a 30 people dataset

to do:
- make sure it's all in caps when needed and the replacement is efficient
- correct the punctuation

# Ethnic experiment

For the ethnic evaluation, cases were ethnicity is non-relevant were selected, so no gynecology, etc.